# Install Dependencies

# Imports

In [ ]:
import tensorflow as tf
print(tf.__version__)
import torch
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

import os
import gc
import cv2
import glob
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
from shutil import copyfile
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

# Load Submission 

In [ ]:
# Read the submisison file
sub_df = pd.read_csv('/kaggle/input/siim-covid19-detection/sample_submission.csv')
print(len(sub_df))
sub_df

In [ ]:
study_df = sub_df.loc[sub_df.id.str.contains('_study')]
len(study_df)

In [ ]:
image_df = sub_df.loc[sub_df.id.str.contains('_image')]
len(image_df)

# Utils to extract images and resize them

In [ ]:

def read_xray(path, voi_lut = True, fix_monochrome = True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data

def resize_xray(array, size, keep_ratio=False, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    
    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    
    return im

In [ ]:
TEST_PATH = f'/kaggle/tmp/test/'
IMG_SIZE = 1024

def prepare_test_images():
    image_id = []
    dim0 = []
    dim1 = []

    os.makedirs(TEST_PATH, exist_ok=True)

    for dirname, _, filenames in tqdm(os.walk(f'../input/siim-covid19-detection/test')):
        for file in filenames:
            # set keep_ratio=True to have original aspect ratio
            xray = read_xray(os.path.join(dirname, file))
            im = resize_xray(xray, size=IMG_SIZE)  
            im.save(os.path.join(TEST_PATH, file.replace('dcm', 'png')))

            image_id.append(file.replace('.dcm', ''))
            dim0.append(xray.shape[0])
            dim1.append(xray.shape[1])
            
    return image_id, dim0, dim1

# Prepare Image Level Test Images

# YOLOv5 Inferencecontains

In [ ]:
YOLO_MODEL_PATH = '../input/happyhappyyolov5xtrain/best.pt'
TEST_PATH = '../input/siim-covide-1024-resized-happyhappy/test'
!python ../input/kaggle-yolov5/detect.py --weights {YOLO_MODEL_PATH} \
                                      --source {TEST_PATH} \
                                      --img {IMG_SIZE} \
                                      --conf 0.16  \
                                      --iou-thres 0.5 \
                                      --max-det 10 \
                                      --save-txt \
                                      --save-conf

In [ ]:
PRED_PATH = 'runs/detect/exp2/labels'
prediction_files = os.listdir(PRED_PATH)
print(f'Number of opacity predicted by YOLOv5: {len(prediction_files)}')

In [ ]:

# The submisison requires xmin, ymin, xmax, ymax format. 
# YOLOv5 returns x_center, y_center, width, height
def correct_bbox_format(bboxes):
    correct_bboxes = []
    for b in bboxes:
        xc, yc = int(np.round(b[0]*IMG_SIZE)), int(np.round(b[1]*IMG_SIZE))
        w, h = int(np.round(b[2]*IMG_SIZE)), int(np.round(b[3]*IMG_SIZE))

        xmin = xc - int(np.round(w/2))
        xmax = xc + int(np.round(w/2))
        ymin = yc - int(np.round(h/2))
        ymax = yc + int(np.round(h/2))
        
        correct_bboxes.append([xmin, xmax, ymin, ymax])
        
    return correct_bboxes

# Read the txt file generated by YOLOv5 during inference and extract 
# confidence and bounding box coordinates.
def get_conf_bboxes(file_path):
    confidence = []
    bboxes = []
    with open(file_path, 'r') as file:
        for line in file:
            preds = line.strip('\n').split(' ')
            preds = list(map(float, preds))
            confidence.append(preds[-1])
            bboxes.append(preds[1:-1])
    return confidence, bboxes

# Read the submisison file
sub_df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
sub_df.tail()

# Prediction loop for submission
predictions = []

for i in tqdm(range(len(sub_df))):
    row = sub_df.loc[i]
    id_name = row.id.split('_')[0]
    id_level = row.id.split('_')[-1]
    
    if id_level == 'study':
        # do study-level classification
        predictions.append("Negative 1 0 0 1 1") # dummy prediction
        
    elif id_level == 'image':
        # we can do image-level classification here.
        # also we can rely on the object detector's classification head.
        # for this example submisison we will use YOLO's classification head. 
        # since we already ran the inference we know which test images belong to opacity.
        if f'{id_name}.txt' in prediction_files:
            # opacity label
            confidence, bboxes = get_conf_bboxes(f'{PRED_PATH}/{id_name}.txt')
            bboxes = correct_bbox_format(bboxes)
            pred_string = ''
            for j, conf in enumerate(confidence):
                pred_string += f'opacity {conf} ' + ' '.join(map(str, bboxes[j])) + ' '
            predictions.append(pred_string[:-1]) 
        else:
            predictions.append("None 1 0 0 1 1")



sub_df['PredictionString'] = predictions

In [ ]:
sub_df

In [ ]:
!rm -rf ./runs/detect

In [ ]:
# study_df = pd.read_csv('../input/siimhappyhappysubmission/study_submission.csv')
# study_df = study_df[study_df.apply(lambda x : x['id'].endswith("study"),axis=1)]

In [ ]:
# # image_df = pd.read_csv('image_submission.csv')
# image_df = image_df[image_df.apply(lambda x : x['id'].endswith("image"),axis=1)]

In [ ]:
# sub_df = pd.concat([study_df, image_df])
sub_df.to_csv('./submission.csv', index=False)
sub_df